1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

**Ответ.** Предпочтительнее обычно precision@k. Потому что нам хочется знать не только сам факт взаимодействия, а какой % рекомендованых товаров юзер купил. Также precision@k даёт в принципе больше информации, ведь зная precision@k мы автоматически знаем значение hit_rate@k, а наоборот так не работает.
<br><br>
hit_rate@k уместно использовать при k=1 или при k>1, когда нам важен только факт взаимодействия. 
<br>
Примеры:<br>
А) В письме e-mail-рассылки с "горящими турами". Клиент - постоянный покупатель, товар - спецпредложение. Т.к. он вряд ли будет брать более 1 тура за раз, то нас интересует, что он купит любой из предложенных.
<br>
Б) По такой же логике можно привести в пример краткосрочную аренду на Booking. Клиент - пользователь, товар - различные апартаменты. Как правило из всей подборки будет забронирован какой-то один вариант.

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



**Ответ**. Логарифм используется для плавного снижения веса, т.е. значимости в зависимости от позиции. При этом функция логарифма обладает таким свойством, что скорость снижения веса сначала максимальная и далее снижается. Можно использовать также логарифмы по разным основаниям. Из других функций - линейная функция ($x$), степенная функция ($x^2$) или экспонента ($e^x$), но веса будут снижаться не так плавно.<br>
Логарифм мы уже встречали в log loss и ещё он есть в TF-IDF

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

**Ответ.** Т.к. можно посчитать Presicion и Recall, то полагаю можно и вычислить AUC PR и также F1-Measure. Интеракции должны быть явными - лайки, простановка рейтинга и т.п. Также есть метрики - Spearman Correlation, Fraction of Concordance Pairs.

Ещё нужно отметить, что некоторые метрики считаются только на бинарных значениях - это, например, все метрики в основе которых presicion/recall.

Есть модели учитывающие последовательный просмотр рекомендаций и вероятность, что пользователь в какой-то момент не продолжит смотреть дальше - каскадные метрики. Особенно ярко такое поведение проявляется в поисковой выдаче Google/Yandex и т.п. Примеры метрик - Expected reciprocal rank (ERR), PFound.

4. bought = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [46]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

In [47]:
bought = np.array([1, 3, 5, 7, 9, 11])
recommended = np.array([2, 5, 7, 4, 11, 9, 8, 10, 12, 3])

In [48]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

def ap_k(recommended_list, bought_list, k=5):
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

def recall_at_k(recommended_list, bought_list, k=5):
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

def reciprocal_rank(recommended_list, bought_list, k=5):
    
    recommended_list = recommended_list[:k]
    ranks = 0
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
                
    return ranks / len(recommended_list)

def normalized_discounted_cumulative_gain(recommended_list, bought_list, k=5):
    
    dcg, idcg = 0, 0
    dcg_list = []
    idcg_list = []
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    for i in range(len(flags)):
        if i == 0:
            dcg_list.append(flags[i]/1)
            idcg_list.append(1/1)
        else:
            dcg_list.append(flags[i]/np.log2(i+1))
            idcg_list.append(1/np.log2(i+1))
    
    return np.mean(np.array(dcg_list)) / np.mean(np.array(idcg_list))

$recommended@8 = [2, 5, 7, 4, 11, 9, 8, 10]$  
$bought = [1, 3, 5, 7, 9, 11]$

$recommended_{mask} = [0, 1, 1, 0, 1, 1, 0, 0]$ 
<br>
$pr@8 = 4 / 8 = 0.5$

In [49]:
# pr@8
precision_at_k(recommended, bought, k=8)

0.5

$recommended@8 = [2, 5, 7, 4, 11, 9, 8, 10]$  
$bought = [1, 3, 5, 7, 9, 11]$

$recommended_{mask} = [0, 1, 1, 0, 1, 1, 0, 0]$ 
<br>
$pr@2 = 1 / 2$<br>
$pr@3 = 2 / 3$<br>
$pr@5 = 3 / 5$<br>
$pr@6 = 4 / 6 = 2 / 3$<br>
$AP@8 = (1/2 + 2/3 + 3/5 + 2/3) / 8 = 0.3041666666666667$

In [50]:
# AP@8
ap_k(recommended, bought, k=8)

0.30416666666666664

$recommended@8 = [2, 5, 7, 4, 11, 9, 8, 10]$  
$bought = [1, 3, 5, 7, 9, 11]$

$bought_{mask} = [0, 0, 1, 1, 1, 1]$ 
<br>
$rec@8 = 4 / 6 = 0.66(6)$

In [51]:
# rec@8
recall_at_k(recommended, bought, k=8)

0.6666666666666666

$recommended@8 = [2, 5, 7, 4, 11, 9, 8, 10]$  
$bought = [1, 3, 5, 7, 9, 11]$

$recommended_{mask} = [0, 1, 1, 0, 1, 1, 0, 0]$ 
<br>
$rec@8 = 1/8 * (1/2 + 1/3 + 1/5 + 1/6) = 0.15$

In [52]:
# MRR@8
reciprocal_rank(recommended, bought, k=8)

0.15

$recommended@8 = [2, 5, 7, 4, 11, 9, 8, 10]$  
$bought = [1, 3, 5, 7, 9, 11]$

$recommended_{mask} = [0, 1, 1, 0, 1, 1, 0, 0]$ 

$DCG@8 = \frac{1}{8}*(0 / 1 + 1 / log_{2}(2) + 1 / log_{2}(3) + 0 / log_{2}(4) + 1 / log_{2}(5) + 1 / log_{2}(6) + 0 / log_{2}(7) + 0 / log_{2}(8))$  
$ideal DCG@8 = \frac{1}{8}*(1 / 1 + 1 / log_{2}(2) + 1 / log_{2}(3) + 1 / log_{2}(4) + 1 / log_{2}(5) + 1 / log_{2}(6) + 1 / log_{2}(7) + 1 / log_{2}(8))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [54]:
# NDCG@8
normalized_discounted_cumulative_gain(recommended, bought, k=8)

0.527912744563727